In [1]:
import os
from PIL import Image
import numpy as np
import random

import cv2
from matplotlib import pyplot as plt

import albumentations as A
import imgaug as ia
import imgaug.augmenters as iaa

In [2]:
fg_root = '/mnt/data-ssd/wa/OneToMany_ToolSynSeg/data_gen/srcdata/fg/endovis18'


In [9]:
# extract mask from original mask
folder_list = sorted(os.listdir(os.path.join(fg_root, 'source')))

for i in range(len(folder_list)):
    mask_dir = os.path.join(fg_root, 'source', folder_list[i], 'mask')
    for mask_name in os.listdir(mask_dir):
        mask = np.array(Image.open(os.path.join(mask_dir, mask_name)))
        class_id = i+1
        # print(np.unique(mask))
        mask[mask!=class_id] = 0
        # print(np.unique(mask))
        mask_pl = Image.fromarray(mask)
        mask_pl.save(os.path.join(fg_root, 'source', folder_list[i], 'mask_pos', mask_name))


In [ ]:
# extract tool based on its mask (image_2)
folder_list = sorted(os.listdir(os.path.join(fg_root, 'source')))
for i in range(len(folder_list)):
    image_dir = os.path.join(fg_root, 'source', folder_list[i], 'image_2')
    mask_dir = os.path.join(fg_root, 'source', folder_list[i], 'mask_pos')
    for image_name in os.listdir(image_dir):
        image_np = np.array(Image.open(os.path.join(image_dir, image_name)).convert('RGBA'))
        mask_np = np.array(Image.open(os.path.join(mask_dir, image_name)).convert('RGBA'))
        image_np[mask_np[:,:,:]==0] = 255
        img = Image.fromarray(image_np)
        
        datas = img.getdata()

        newData = []
        for items in datas:
            if items[0] == 255 and items[1] == 255 and items[2] == 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(items)

        img.putdata(newData)
        img.save(os.path.join(fg_root, 'source', folder_list[i], 'image_2_pos', image_name), "PNG")
        

In [3]:
# extract tool based on its mask (image_3)
folder_list = sorted(os.listdir(os.path.join(fg_root, 'source')))
for i in range(len(folder_list)):
    image_dir = os.path.join(fg_root, 'source', folder_list[i], 'image_3')
    mask_dir = os.path.join(fg_root, 'source', folder_list[i], 'mask_pos')
    for image_name in os.listdir(image_dir):
        image_np = np.array(Image.open(os.path.join(image_dir, image_name)).convert('RGBA'))
        mask_np = np.array(Image.open(os.path.join(mask_dir, image_name)).convert('RGBA'))
        image_np[mask_np[:,:,:]==0] = 255
        img = Image.fromarray(image_np)
        
        datas = img.getdata()

        newData = []
        for items in datas:
            if items[0] == 255 and items[1] == 255 and items[2] == 255:
                newData.append((255, 255, 255, 0))
            else:
                newData.append(items)

        img.putdata(newData)
        img.save(os.path.join(fg_root, 'source', folder_list[i], 'image_3_pos', image_name), "PNG")
        

In [56]:
def visualize(image, mask, original_image=None, original_mask=None):
    fontsize = 18
    
    if original_image is None and original_mask is None:
        f, ax = plt.subplots(2, 1, figsize=(8, 8))

        ax[0].imshow(image)
        ax[1].imshow(mask)
    else:
        f, ax = plt.subplots(2, 2, figsize=(8, 8))

        ax[0, 0].imshow(original_image)
        ax[0, 0].set_title('Original image', fontsize=fontsize)
        
        ax[1, 0].imshow(original_mask)
        ax[1, 0].set_title('Original mask', fontsize=fontsize)
        
        ax[0, 1].imshow(image)
        ax[0, 1].set_title('Transformed image', fontsize=fontsize)
        
        ax[1, 1].imshow(mask)
        ax[1, 1].set_title('Transformed mask', fontsize=fontsize)

In [4]:
def sometimes_1(aug): return iaa.Sometimes(0.6, aug)
def sometimes_2(aug): return iaa.Sometimes(0.2, aug)
def sometimes_3(aug): return iaa.Sometimes(0.4, aug)
seq = iaa.Sequential([
    iaa.Fliplr(0.5),  # horizontally flip 50% of all images
    iaa.Flipud(0.2),  # vertically flip 20% of all images
    sometimes_2(iaa.Crop(px=(10, 200))),
    # improve or worsen the contrast
    # sometimes_1(iaa.LinearContrast((0.5, 2.0))), # +
    # sometimes_1(iaa.LinearContrast((0.5, 2.0), per_channel=0.5)),
    sometimes_2(iaa.PerspectiveTransform(scale=(0.1, 0.2))), # +
    sometimes_1(iaa.Affine(
        # scale images to 80-120% of their size, individually per axis
        scale={"x": (0.9, 1.1), "y": (0.9, 1.1)},
        # translate by -20 to +20 percent (per axis)
        translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)},
        rotate=(-45, 45),  # rotate by -45 to +45 degrees
        # shear=(-16, 16), # shear by -16 to +16 degrees
        order=[0, 1],  # use nearest neighbour or bilinear interpolation (fast)
        # cval=(0, 255), # if mode is constant, use a cval between 0 and 255
        cval=0,
        # use any of scikit-image's warping modes (see 2nd image from the top for examples)
        mode='constant'
    )),
    # sometimes_1(iaa.OneOf([
    #         # iaa.Multiply((0.5, 1.5), per_channel=0.5),
    #         iaa.Multiply((0.5, 1.5)),
    #         iaa.FrequencyNoiseAlpha(
    #             exponent=(-4, 0),
    #             # first=iaa.Multiply((0.5, 1.5), per_channel=True),
    #             first=iaa.Multiply((0.5, 1.5)),
    #             second=iaa.LinearContrast((0.5, 2.0))
    #         )
    #     ])), # +
    # iaa.Multiply((0.5, 1.5), per_channel=0.5), # +
    # iaa.SimplexNoiseAlpha(iaa.OneOf([
    #                 iaa.EdgeDetect(alpha=(0, 0.5)),
    #                 iaa.DirectedEdgeDetect(alpha=(0, 0.5), direction=(0.0, 1.0)),
    #             ])),
    # sometimes_1(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)), # move pixels locally around (with random strengths)

    sometimes_3(iaa.OneOf([
        # blur images with a sigma between 0 and 3.0
        iaa.GaussianBlur((0, 3.0)),
        # blur image using local means with kernel sizes between 2 and 7
        iaa.AverageBlur(k=(2, 7)),
        # blur image using local medians with kernel sizes between 2 and 7
        iaa.MedianBlur(k=(3, 11)),
    ])),
],random_order=True)

In [55]:
image1 = np.array(Image.open('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/source/1-Bipolar_Forceps/image_2_pos/seq_4_frame003.png').convert('RGB'), dtype=np.uint8)
image2 = np.array(Image.open('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/source/1-Bipolar_Forceps/image_2_pos/seq_4_frame071.png').convert('RGB'), dtype=np.uint8)
images = []
images.append(image1)
images.append(image2)
# [image1, image2]
# print(image2.shape)
# rotate = iaa.Affine(rotate=(-25, 25))
# print(np.array(images).shape)
# images = np.random.randint(0, 255, (16, 128, 128, 3), dtype=np.uint8)
images_aug = seq(images=images)
# ia.imshow(images_aug[0])
# print(len(images_aug))

In [74]:
# 2 base image per instru
from tqdm import tqdm
import sys
N = 50 # number of variants for each image/mask
folder_list = sorted(os.listdir(os.path.join(fg_root, 'source')))
# print(folder_list)
# sys.exit()
for folder in folder_list[:]:
    image_list = os.listdir(os.path.join(fg_root, 'source', folder, 'image_2'))
    # print(image_list)
    for img_name in image_list:
        # image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_pos', img_name)), dtype=np.uint8)
        image = np.array(Image.open(os.path.join(fg_root, 'source', folder, 'image_2', img_name)).convert('RGB'), dtype=np.uint8)
        # print(image.shape)
        # sys.exit(0)
        segmap = np.array(Image.open(os.path.join(fg_root, 'source', folder, 'mask_pos', img_name)), dtype=np.int32)
        images = np.expand_dims(image, axis=0)
        segmaps = np.expand_dims(segmap, axis=0)
        segmaps = np.expand_dims(segmaps, axis=3)

        for i in tqdm(range(N), ncols=100, desc=folder+'-'+img_name):
            images_aug, segmaps_aug = seq(images=images, segmentation_maps=segmaps)
            images_aug = np.squeeze(images_aug)
            segmaps_aug = np.squeeze(segmaps_aug)
            segmaps_aug = segmaps_aug.astype('uint8')
            
            image_pil = Image.fromarray(images_aug).convert('RGBA')
            mask_pil = Image.fromarray(segmaps_aug).convert('RGBA')

            image_np = np.array(image_pil)
            mask_np = np.array(mask_pil)

            image_np[mask_np[:,:,:]==0] = 255
            img = Image.fromarray(image_np)
            datas = img.getdata()
            newData = []
            for items in datas:
                if items[0] == 255 and items[1] == 255 and items[2] == 255:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(items)
            img.putdata(newData)
            # img.save(os.path.join(folder_list[i], 'image_pos', image_name), "PNG")

            msk = Image.fromarray(segmaps_aug)
            # visualize(img, msk)
            # sys.exit()
            img.save('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/aug/'+ folder + '/image_2_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            msk.save('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/aug/'+ folder + '/mask_2_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            # sys.exit()

7-Clip_Applier-seq_1_frame033.png: 100%|████████████████████████████| 50/50 [00:27<00:00,  1.85it/s]


In [81]:
# binarize mask: mask[mask!=0] == 255
folder_list = sorted(os.listdir(os.path.join(fg_root, 'aug')))
for folder in folder_list[:]:
    mask_list = os.listdir(os.path.join(fg_root, 'aug', folder, 'mask_2_50'))
    for mask_name in mask_list:
        mask_np = np.array(Image.open(os.path.join(fg_root, 'aug', folder, 'mask_2_50', mask_name)))
        # print(np.unique(mask_np))
        mask_np[mask_np!=0] = 255
        # print(np.unique(mask_np))
        mask = Image.fromarray(mask_np)
        mask.save(os.path.join(fg_root, 'aug', folder, 'mask_2_50_bin', mask_name))
        # sys.exit()

In [6]:
# 3 base image per instru
from tqdm import tqdm
import sys
N = 50 # number of variants for each image/mask
folder_list = sorted(os.listdir(os.path.join(fg_root, 'source')))
# print(folder_list)
# sys.exit()
for folder in folder_list[:]:
    image_list = os.listdir(os.path.join(fg_root, 'source', folder, 'image_3'))
    # print(image_list)
    for img_name in image_list:
        # image = np.array(Image.open(os.path.join('/mnt/data-ssd/wa/overlay/FgBgAug/fg/EndoVis18/', folder, 'image_pos', img_name)), dtype=np.uint8)
        image = np.array(Image.open(os.path.join(fg_root, 'source', folder, 'image_3', img_name)).convert('RGB'), dtype=np.uint8)
        # print(image.shape)
        # sys.exit(0)
        segmap = np.array(Image.open(os.path.join(fg_root, 'source', folder, 'mask_pos', img_name)), dtype=np.int32)
        images = np.expand_dims(image, axis=0)
        segmaps = np.expand_dims(segmap, axis=0)
        segmaps = np.expand_dims(segmaps, axis=3)

        for i in tqdm(range(N), ncols=100, desc=folder+'-'+img_name):
            images_aug, segmaps_aug = seq(images=images, segmentation_maps=segmaps)
            images_aug = np.squeeze(images_aug)
            segmaps_aug = np.squeeze(segmaps_aug)
            segmaps_aug = segmaps_aug.astype('uint8')
            
            image_pil = Image.fromarray(images_aug).convert('RGBA')
            mask_pil = Image.fromarray(segmaps_aug).convert('RGBA')

            image_np = np.array(image_pil)
            mask_np = np.array(mask_pil)

            image_np[mask_np[:,:,:]==0] = 255
            img = Image.fromarray(image_np)
            datas = img.getdata()
            newData = []
            for items in datas:
                if items[0] == 255 and items[1] == 255 and items[2] == 255:
                    newData.append((255, 255, 255, 0))
                else:
                    newData.append(items)
            img.putdata(newData)
            # img.save(os.path.join(folder_list[i], 'image_pos', image_name), "PNG")

            msk = Image.fromarray(segmaps_aug)
            # visualize(img, msk)
            # sys.exit()
            img.save('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/aug/'+ folder + '/image_3_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            msk.save('/mnt/data-ssd/wa/miccai_ext/data_gen/srcdata/fg/endovis18/aug/'+ folder + '/mask_3_50/' + img_name[:-4] + '_' + str(i)+'.png', 'PNG')
            # sys.exit()

7-Clip_Applier-seq_1_frame043.png: 100%|████████████████████████████| 50/50 [00:23<00:00,  2.09it/s]


In [ ]:
# binarize mask: mask[mask!=0] == 255
folder_list = sorted(os.listdir(os.path.join(fg_root, 'aug')))
for folder in folder_list[:]:
    mask_list = os.listdir(os.path.join(fg_root, 'aug', folder, 'mask_3_50'))
    for mask_name in mask_list:
        mask_np = np.array(Image.open(os.path.join(fg_root, 'aug', folder, 'mask_3_50', mask_name)))
        # print(np.unique(mask_np))
        mask_np[mask_np!=0] = 255
        # print(np.unique(mask_np))
        mask = Image.fromarray(mask_np)
        mask.save(os.path.join(fg_root, 'aug', folder, 'mask_3_50_bin', mask_name))
        # sys.exit()